<a href="https://colab.research.google.com/github/chaiminwoo0223/Deep-Learning/blob/main/12%20-%20RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
n_hidden = 35
lr = 0.01
epochs = 1000

In [3]:
# 사용하는 문자는 영어 소문자 및 몇가지 특수문자로 제한한다.
# alphabet(0-25), space(26), ... , start(0), end(1)

string = "i don't want a perfect life, i want a happy life!"
chars = "abcdefghijklmnopqrstuvwxyz '!.,:;01"

char_list = [i for i in chars] # 문자열을 리스트로 바꾼다.
n_letters = len(char_list)     # 문자의 개수를 저장한다.(문자열의 길이)

In [4]:
print(char_list)
print(n_letters)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', "'", '!', '.', ',', ':', ';', '0', '1']
35


In [5]:
# 문자를 그대로 쓰지 않고, one-hot 벡터로 바꿔서 연산한다.

# start = [0 0 0 … 1 0]
# a =     [1 0 0 … 0 0]
# b =     [0 1 0 … 0 0]
# c =     [0 0 1 … 0 0]
# …
# end =   [0 0 0 … 0 1]

In [6]:
# 문자열을 one-hot 벡터의 스텍으로 만드는 함수
def string_to_onehot(string):
    # 시작 토큰과 끝 토큰을 만든다.
    start = np.zeros(shape=n_letters, dtype=int) # [0 0 0 … 0 0]
    end = np.zeros(shape=n_letters, dtype=int)   # [0 0 0 … 0 0]
    start[-2] = 1                                # [0 0 0 … 1 0]
    end[-1] = 1                                  # [0 0 0 … 0 1]

    for i in string:
        # 문자가 몇번째 문자인지 찾는다.(a:0, b:1, c:2, ...)
        idx = char_list.index(i)
        # 0으로만 구성된 배열을 만들어준다.([0 0 0 … 0 0])
        zero = np.zeros(shape=n_letters, dtype=int)
        # 해당 문자 인덱스만 1로 바꾼다.([1 0 0 … 0 0])
        zero[idx] = 1
        # start와 새로 생긴 zero를 붙이고, 이를 start에 할당한다.
        # 이것이 반복되면, start에는 문자를 one-hot 벡터로 바꾼 배열들이 점점 쌓여가게 된다.
        start = np.vstack([start,zero])
    # 문자열이 다 끝나면, 쌓아온 start와 end를 붙인다.
    output = np.vstack([start,end])
    return output

In [7]:
# one-hot 벡터를 문자로 바꿔주는 함수
def onehot_to_word(onehot_1):
    # 텐서를 입력으로 받아, 넘파이 배열로 바꾼다.
    onehot = torch.Tensor.numpy(onehot_1)
    # one-hot 벡터의 최댓값(=1) 위치 인덱스로 문자를 찾습니다.
    return char_list[onehot.argmax()]

In [8]:
# RNN
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.act_fn = nn.Tanh()

    def forward(self, input_, hidden):
        combined = torch.cat((input_, hidden), 1)
        hidden = self.act_fn(self.i2h(combined)) # hidden state 업데이트
        output = self.i2o(combined)              # output
        return output, hidden

    def init_hidden(self):
        return torch.zeros(1, self.hidden_size) # 아직 입력이 없을 때(t=0)의 hidden state를 초기화

rnn = RNN(n_letters, n_hidden, n_letters)

In [9]:
# 손실함수와 최적화
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

In [10]:
# Train
one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor()) # 문자열 -> onehot 벡터 ->.토치 텐서

for i in range(epochs):
    optimizer.zero_grad()
    hidden = rnn.init_hidden() # 학습에 앞서, hidden state를 초기화
    total_loss = 0

    for j in range(one_hot.size()[0]-1):
        input_ = one_hot[j:j+1,:] # 입력 = 앞글자(h e l l)
        target = one_hot[j+1]    # 타겟 = 뒷글자(e l l o)
        output, hidden = rnn.forward(input_, hidden)
        loss = loss_func(output.view(-1), target.view(-1))
        total_loss += loss

    total_loss.backward()
    optimizer.step()
    if i % 10 == 0:
        print(total_loss)

tensor(1.9194, grad_fn=<AddBackward0>)
tensor(0.6693, grad_fn=<AddBackward0>)
tensor(0.3048, grad_fn=<AddBackward0>)
tensor(0.1727, grad_fn=<AddBackward0>)
tensor(0.1030, grad_fn=<AddBackward0>)
tensor(0.0761, grad_fn=<AddBackward0>)
tensor(0.0652, grad_fn=<AddBackward0>)
tensor(0.0608, grad_fn=<AddBackward0>)
tensor(0.0586, grad_fn=<AddBackward0>)
tensor(0.0601, grad_fn=<AddBackward0>)
tensor(0.0575, grad_fn=<AddBackward0>)
tensor(0.0541, grad_fn=<AddBackward0>)
tensor(0.0477, grad_fn=<AddBackward0>)
tensor(0.0412, grad_fn=<AddBackward0>)
tensor(0.0367, grad_fn=<AddBackward0>)
tensor(0.0309, grad_fn=<AddBackward0>)
tensor(0.0284, grad_fn=<AddBackward0>)
tensor(0.0309, grad_fn=<AddBackward0>)
tensor(0.0278, grad_fn=<AddBackward0>)
tensor(0.0254, grad_fn=<AddBackward0>)
tensor(0.0228, grad_fn=<AddBackward0>)
tensor(0.0190, grad_fn=<AddBackward0>)
tensor(0.0243, grad_fn=<AddBackward0>)
tensor(0.0153, grad_fn=<AddBackward0>)
tensor(0.0161, grad_fn=<AddBackward0>)
tensor(0.0084, grad_fn=<A

In [11]:
# Test
start = torch.zeros(1, n_letters)
start[:, -2] = 1

with torch.no_grad():
    hidden = rnn.init_hidden()
    input_ = start
    output_string = ""

    for i in range(len(string)):
        output, hidden = rnn.forward(input_, hidden)
        output_string += onehot_to_word(output.data)
        input_ = output

print(output_string)

i don't want a perfect life, i want a hapeyy lrfa


In [12]:
start = np.zeros(shape=n_letters, dtype=int)
start[-2] = 1
start

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [13]:
start = torch.zeros(1, n_letters)
start[:, -2] = 1
start

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]])